In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/FILE LATIHAN PYTHON/flask-sqlite/flask-sqlite-login/")

!pwd

/content/drive/My Drive/FILE LATIHAN PYTHON/flask-sqlite/flask-sqlite-login


In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

     |████████████████████████████████| 83 kB 710 kB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [ ]:
# !pip install flask
!pip install SQLAlchemy
!pip install Flask-SQLAlchemy

In [ ]:
# flask_ngrok_example.py
from flask import Flask
#from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return "Hello FGA Big Data Using Python Kelas B"

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://34d8bf7acd48.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Aug/2021 16:58:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2021 16:58:57] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# flask_ngrok_example.py
from flask import Flask,render_template,flash, redirect,url_for,session,logging,request
from flask_sqlalchemy import SQLAlchemy
#from flask_ngrok import run_with_ngrok

# !pip install SQLAlchemy
# !pip install Flask-SQLAlchemy

# app = Flask(__name__)
# app = Flask(__name__, static_url_path='/static')
app = Flask(__name__, static_folder='static')
#run_with_ngrok(app)  # Start ngrok when app is run

# app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////fga_big_data.db'
db = SQLAlchemy(app)

class user(db.Model):
  # __tablename__ = 'user'
  id = db.Column(db.Integer, primary_key=True)
  # Mail 	Password 	Name 	Level
  Mail = db.Column(db.Text) # sbg Username
  Password = db.Column(db.Text)
  Name = db.Column(db.Text)
  Level = db.Column(db.Text)

  # username = Mail
  # password = Password

@app.route("/")
def index():
    
    return render_template("index.html")

@app.route("/signout")
def sign_out():
    session.pop('name')
    session.pop('mail')
    return redirect(url_for("index"))

@app.route("/login",methods=["GET", "POST"])
def login():
    msg = ""
    if request.method == "POST":
        mail = request.form["mail"]
        passw = request.form["passw"]
        
        login = user.query.filter_by(Mail=mail, Password=passw).first()
        print(login)
        if login is not None:
            # return redirect(url_for("index"))
            session['name'] = login.Name
            session['mail'] = login.Mail
            # return render_template('bigdataApps.html', login=login)

            return redirect(url_for("bigdataApps"))
        elif login is None:
            msg = "Masukkan Username (Email) dan Password dgn Benar"
            print(msg)

    return render_template("login.html", msg = msg)

@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST":
        mail = request.form['mail']
        uname = request.form['uname']
        passw = request.form['passw']

        register = user(Mail = mail, Name = uname, Password = passw)
        # register = user(Mail = mail, Password = passw)
        db.session.add(register)
        db.session.commit()

        return redirect(url_for("login"))
    return render_template("register.html")

@app.route("/bigdataApps", methods=["GET", "POST"])
def bigdataApps():
  if request.method == 'POST':
    import pandas as pd
    import numpy as np
    dataset = request.FILES['inputDataset']
    # dataset = request.files['inputDataset']
    #dataset = 'dataset_dump.csv'

    persentase_data_training = 90
    banyak_fitur = int(request.POST['banyakFitur'])
    banyak_hidden_neuron = int(request.POST['banyakHiddenNeuron'])
    dataset = pd.read_csv(dataset, delimiter=';', names = ['Tanggal', 'Harga'], usecols=['Harga'])
    minimum = int(dataset.min()-10000)
    maksimum = int(dataset.max()+10000)
    new_banyak_fitur = banyak_fitur + 1
    hasil_fitur = []
    for i in range((len(dataset)-new_banyak_fitur)+1):
      kolom = []
      j = i
      while j < (i+new_banyak_fitur):
        kolom.append(dataset.values[j][0])
        j += 1
      hasil_fitur.append(kolom)
    hasil_fitur = np.array(hasil_fitur)
    data_normalisasi = (hasil_fitur - minimum)/(maksimum - minimum)
    data_training = data_normalisasi[:int(persentase_data_training*len(data_normalisasi)/100)]
    data_testing = data_normalisasi[int(persentase_data_training*len(data_normalisasi)/100):]
    
    #Training
    bobot = np.random.rand(banyak_hidden_neuron, banyak_fitur)
    bias = np.random.rand(banyak_hidden_neuron)
    h = 1/(1 + np.exp(-(np.dot(data_training[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    h_plus = np.dot(np.linalg.inv(np.dot(np.transpose(h),h)),np.transpose(h))
    output_weight = np.dot(h_plus, data_training[:, banyak_fitur])
    
    #Testing
    h = 1/(1 + np.exp(-(np.dot(data_testing[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    predict = np.dot(h, output_weight)
    predict = predict * (maksimum - minimum) + minimum
    
    #MAPE
    aktual = np.array(hasil_fitur[int(persentase_data_training*len(data_normalisasi)/100):, banyak_fitur])
    mape = np.sum(np.abs(((aktual - predict)/aktual)*100))/len(predict)
    return render_template('bigdataApps.html', {
        'y_aktual' : list(aktual),
        'y_prediksi' : list(predict),
        'mape' : mape
        })
  else:
    return render_template('bigdataApps.html')

if __name__ == '__main__':
  db.create_all()
  app.secret_key = 'webapp'
  app.run()

In [3]:
%%file Tugas4_webapp.py

# flask_ngrok_example.py
from flask import Flask,render_template,flash, redirect,url_for,session,logging,request
from flask_sqlalchemy import SQLAlchemy
#from flask_ngrok import run_with_ngrok

# !pip install SQLAlchemy
# !pip install Flask-SQLAlchemy

# app = Flask(__name__)
# app = Flask(__name__, static_url_path='/static')
app = Flask(__name__, static_folder='static')
#run_with_ngrok(app)  # Start ngrok when app is run

# app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////fga_big_data.db'
db = SQLAlchemy(app)

class user(db.Model):
  # __tablename__ = 'user'
  id = db.Column(db.Integer, primary_key=True)
  # Mail 	Password 	Name 	Level
  Mail = db.Column(db.Text) # sbg Username
  Password = db.Column(db.Text)
  Name = db.Column(db.Text)
  Level = db.Column(db.Text)

  # username = Mail
  # password = Password

@app.route("/")
def index():
    
    return render_template("index.html")

@app.route("/signout")
def sign_out():
    session.pop('name')
    session.pop('mail')
    return redirect(url_for("index"))

@app.route("/login",methods=["GET", "POST"])
def login():
    msg = ""
    if request.method == "POST":
        mail = request.form["mail"]
        passw = request.form["passw"]
        
        login = user.query.filter_by(Mail=mail, Password=passw).first()
        print(login)
        if login is not None:
            # return redirect(url_for("index"))
            session['name'] = login.Name
            session['mail'] = login.Mail
            # return render_template('bigdataApps.html', login=login)

            return redirect(url_for("bigdataApps"))
        elif login is None:
            msg = "Masukkan Username (Email) dan Password dgn Benar"
            print(msg)

    return render_template("login.html", msg = msg)

@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST":
        mail = request.form['mail']
        uname = request.form['uname']
        passw = request.form['passw']

        register = user(Mail = mail, Name = uname, Password = passw)
        # register = user(Mail = mail, Password = passw)
        db.session.add(register)
        db.session.commit()

        return redirect(url_for("login"))
    return render_template("register.html")

@app.route("/bigdataApps", methods=["GET", "POST"])
def bigdataApps():
  if request.method == 'POST':
    import pandas as pd
    import numpy as np
    dataset = request.FILES['inputDataset']
    # dataset = request.files['inputDataset']
    #dataset = 'dataset_dump.csv'

    persentase_data_training = 90
    banyak_fitur = int(request.POST['banyakFitur'])
    banyak_hidden_neuron = int(request.POST['banyakHiddenNeuron'])
    dataset = pd.read_csv(dataset, delimiter=';', names = ['Tanggal', 'Harga'], usecols=['Harga'])
    minimum = int(dataset.min()-10000)
    maksimum = int(dataset.max()+10000)
    new_banyak_fitur = banyak_fitur + 1
    hasil_fitur = []
    for i in range((len(dataset)-new_banyak_fitur)+1):
      kolom = []
      j = i
      while j < (i+new_banyak_fitur):
        kolom.append(dataset.values[j][0])
        j += 1
      hasil_fitur.append(kolom)
    hasil_fitur = np.array(hasil_fitur)
    data_normalisasi = (hasil_fitur - minimum)/(maksimum - minimum)
    data_training = data_normalisasi[:int(persentase_data_training*len(data_normalisasi)/100)]
    data_testing = data_normalisasi[int(persentase_data_training*len(data_normalisasi)/100):]
    
    #Training
    bobot = np.random.rand(banyak_hidden_neuron, banyak_fitur)
    bias = np.random.rand(banyak_hidden_neuron)
    h = 1/(1 + np.exp(-(np.dot(data_training[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    h_plus = np.dot(np.linalg.inv(np.dot(np.transpose(h),h)),np.transpose(h))
    output_weight = np.dot(h_plus, data_training[:, banyak_fitur])
    
    #Testing
    h = 1/(1 + np.exp(-(np.dot(data_testing[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    predict = np.dot(h, output_weight)
    predict = predict * (maksimum - minimum) + minimum
    
    #MAPE
    aktual = np.array(hasil_fitur[int(persentase_data_training*len(data_normalisasi)/100):, banyak_fitur])
    mape = np.sum(np.abs(((aktual - predict)/aktual)*100))/len(predict)
    return render_template('bigdataApps.html', {
        'y_aktual' : list(aktual),
        'y_prediksi' : list(predict),
        'mape' : mape
        })
  else:
    return render_template('bigdataApps.html')

if __name__ == '__main__':
  db.create_all()
  app.secret_key = 'webapp'
  app.run()

Writing Tugas4_webapp.py


In [ ]:
!pip install Jinja2

In [ ]:
!pip freeze

In [ ]:
%%file ./templates/include/script.html
<!-- Jquery JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/jquery.min.js') }}"></script>
<!-- Jquery JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/jquery.sparkline.min.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/script.js') }}"></script>
<!-- Bootstrap Core JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/bootstrap.min.js') }}"></script>
<!-- Menu Plugin JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/sidebar-nav.min.js') }}"></script>
<!--slimscroll JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/jquery.slimscroll.js') }}"></script>
<!--Wave Effects -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/waves.js') }}"></script>
<!--Morris JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/raphael-min.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/morris.js') }}"></script>
<!-- chartist chart -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/chartist.min.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/chartist-plugin-tooltip.min.js') }}"></script>
<!-- Calendar JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/moment.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/fullcalendar.min.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/cal-init.js') }}"></script>
<!-- Custom Theme JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/custom.min.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/dashboard1.js') }}"></script>
<!-- Custom tab JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/cbpFWTabs.js') }}"></script>
<script type="text/javascript" >
(function() {
    [].slice.call(document.querySelectorAll('.sttabs')).forEach(function(el) {
        new CBPFWTabs(el);
    });
})();
</script>
<!-- Toast JavaScript -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/jquery.toast.js') }}"></script>
<!--Style Switcher -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/jQuery.style.switcher.js') }}"></script>
<!--File Upload -->
<script type="text/javascript" src="{{ url_for('static', filename = 'js/jasny-bootstrap.js') }}"></script>
<script type="text/javascript" src="{{ url_for('static', filename = 'js/highcharts.js') }}"></script>
<script>
Highcharts.chart('hasilGrafik', {
    chart: {
        type: 'spline'
    },
    
    title: {
        text: 'Prediksi Harga Kebutuhan Pokok'
    },

    xAxis: {
        title: {
            text: 'Data ke-'
        }
    },

    yAxis: {
        title: {
            text: 'Harga (Rupiah)'
        }
    },

    legend: {
        layout: 'vertical',
        align: 'right',
        verticalAlign: 'middle'
    },

    plotOptions: {
        series: {
            label: {
                connectorAllowed: false
            }
        }
    },

    series: [{
        name: 'Y Aktual',
        data:  {{ y_aktual }}
    }, {
        name: 'Y Prediksi',
        data: {{ y_prediksi }}
    }],

    responsive: {
        rules: [{
            condition: {
                maxWidth: 500
            },
            chartOptions: {
                legend: {
                    layout: 'horizontal',
                    align: 'center',
                    verticalAlign: 'bottom'
                }
            }
        }]
    }

});
</script>

Overwriting ./templates/include/script.html


In [ ]:
%%file ./templates/include/left_sidebar.html
<div class="navbar-default sidebar" role="navigation">
    <div class="sidebar-nav slimscrollsidebar">
        <div class="sidebar-head">
            <h3><span class="fa-fw open-close"><i class="ti-menu hidden-xs"></i><i class="ti-close visible-xs"></i></span> <span class="hide-menu">Menu</span></h3> </div>
        <ul class="nav" id="side-menu">
            <li class="user-pro">
                <a href="#" class="waves-effect"><img src="{{ url_for('static', filename = 'img/guest.png') }}" alt="user-img" class="img-circle"> <span class="hide-menu">{{session.name}}<span class="fa arrow"></span></span>
                </a>
                <ul class="nav nav-second-level collapse" aria-expanded="false" style="height: 0px;">
                    <li><a href="javascript:void(0)"><i class="ti-user"></i> <span class="hide-menu">{{session.name}}</span></a></li>
                    <li><a href="/signout"><i class="fa fa-power-off"></i> <span class="hide-menu">Logout</span></a></li>
                </ul>
            </li>
            <li class="devider"></li>
            <li> <a href="week1" class="waves-effect"><i class="mdi mdi-numeric-1-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 1</span></a> </li>
            <li> <a href="week21" class="waves-effect"><i class="mdi mdi-numeric-2-box-multiple-outline fa-fw" data-icon="v"></i> <span class="hide-menu"> Week 2 <span class="fa arrow"></span> <span class="label label-rouded label-inverse pull-right">2</span></span></a>
                <ul class="nav nav-second-level">
                    <li> <a href="week21"><i class=" fa-fw">1</i><span class="hide-menu">Task 1</span></a> </li>
                    <li> <a href="week22"><i class=" fa-fw">2</i><span class="hide-menu">Task 2</span></a> </li>
                </ul>
            </li>
            <li> <a href="week3" class="waves-effect"><i class="mdi mdi-numeric-3-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 3</span></a> </li>
            <li> <a href="week4" class="waves-effect"><i class="mdi mdi-numeric-4-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 4</span></a> </li>
            <li> <a href="week5" class="waves-effect"><i class="mdi mdi-numeric-5-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 5</span></a> </li>
            <li> <a href="week6" class="waves-effect"><i class="mdi mdi-numeric-6-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 6</span></a> </li>
            <li> <a href="week7" class="waves-effect"><i class="mdi mdi-numeric-7-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 7</span></a> </li>
            <li> <a href="week8" class="waves-effect"><i class="mdi mdi-numeric-8-box-multiple-outline fa-fw"></i><span class="hide-menu">Week 8</span></a> </li>
        </ul>
    </div>
</div>

Writing ./templates/include/left_sidebar.html


FileNotFoundError: ignored

In [ ]:
%%file ./templates/include/top_header.html
<nav class="navbar navbar-default navbar-static-top m-b-0">
    <div class="navbar-header">
        <div class="top-left-part">
            <a class="logo" href="index.html">
                <b>
                    <img src="{{ url_for('static', filename = 'img/ub.png') }}" alt="home" class="dark-logo" />
                </b>
                <span class="hidden-xs">
                    <img src="{{ url_for('static', filename = 'img/filkom1.png') }}" alt="home" class="dark-logo" />
                </span> 
            </a>
        </div>
        <!-- Search input and Toggle icon -->
        <ul class="nav navbar-top-links navbar-left">
            <li><a href="javascript:void(0)" class="open-close waves-effect waves-light visible-xs"><i class="ti-close ti-menu"></i></a></li>
            <li class="dropdown">
                <a class="dropdown-toggle waves-effect waves-light" data-toggle="dropdown" href="#"> <i class="mdi mdi-gmail"></i>
                    <div class="notify"> <span class="heartbit"></span> <span class="point"></span> </div>
                </a>
                <ul class="dropdown-menu mailbox animated bounceInDown">
                    <li>
                        <div class="drop-title">You have 4 new messages</div>
                    </li>
                    <li>
                        <div class="message-center">
                            <a href="#">
                                <div class="user-img"> <img src="../plugins/images/users/pawandeep.jpg" alt="user" class="img-circle"> <span class="profile-status online pull-right"></span> </div>
                                <div class="mail-contnet">
                                    <h5>Pavan kumar</h5> <span class="mail-desc">Just see the my admin!</span> <span class="time">9:30 AM</span> </div>
                            </a>
                            <a href="#">
                                <div class="user-img"> <img src="../plugins/images/users/sonu.jpg" alt="user" class="img-circle"> <span class="profile-status busy pull-right"></span> </div>
                                <div class="mail-contnet">
                                    <h5>Sonu Nigam</h5> <span class="mail-desc">I've sung a song! See you at</span> <span class="time">9:10 AM</span> </div>
                            </a>
                            <a href="#">
                                <div class="user-img"> <img src="../plugins/images/users/arijit.jpg" alt="user" class="img-circle"> <span class="profile-status away pull-right"></span> </div>
                                <div class="mail-contnet">
                                    <h5>Arijit Sinh</h5> <span class="mail-desc">I am a singer!</span> <span class="time">9:08 AM</span> </div>
                            </a>
                            <a href="#">
                                <div class="user-img"> <img src="../plugins/images/users/pawandeep.jpg" alt="user" class="img-circle"> <span class="profile-status offline pull-right"></span> </div>
                                <div class="mail-contnet">
                                    <h5>Pavan kumar</h5> <span class="mail-desc">Just see the my admin!</span> <span class="time">9:02 AM</span> </div>
                            </a>
                        </div>
                    </li>
                    <li>
                        <a class="text-center" href="javascript:void(0);"> <strong>See all notifications</strong> <i class="fa fa-angle-right"></i> </a>
                    </li>
                </ul>
            </li>
            <!-- .Task dropdown -->
            <li class="dropdown">
                <a class="dropdown-toggle waves-effect waves-light" data-toggle="dropdown" href="#"> <i class="mdi mdi-check-circle"></i>
                    <div class="notify"><span class="heartbit"></span><span class="point"></span></div>
                </a>
                <ul class="dropdown-menu dropdown-tasks animated slideInUp">
                    <li>
                        <a href="#">
                            <div>
                                <p> <strong>Task 1</strong> <span class="pull-right text-muted">40% Complete</span> </p>
                                <div class="progress progress-striped active">
                                    <div class="progress-bar progress-bar-success" role="progressbar" aria-valuenow="40" aria-valuemin="0" aria-valuemax="100" style="width: 40%"> <span class="sr-only">40% Complete (success)</span> </div>
                                </div>
                            </div>
                        </a>
                    </li>
                    <li class="divider"></li>
                    <li>
                        <a href="#">
                            <div>
                                <p> <strong>Task 2</strong> <span class="pull-right text-muted">20% Complete</span> </p>
                                <div class="progress progress-striped active">
                                    <div class="progress-bar progress-bar-info" role="progressbar" aria-valuenow="20" aria-valuemin="0" aria-valuemax="100" style="width: 20%"> <span class="sr-only">20% Complete</span> </div>
                                </div>
                            </div>
                        </a>
                    </li>
                    <li class="divider"></li>
                    <li>
                        <a href="#">
                            <div>
                                <p> <strong>Task 3</strong> <span class="pull-right text-muted">60% Complete</span> </p>
                                <div class="progress progress-striped active">
                                    <div class="progress-bar progress-bar-warning" role="progressbar" aria-valuenow="60" aria-valuemin="0" aria-valuemax="100" style="width: 60%"> <span class="sr-only">60% Complete (warning)</span> </div>
                                </div>
                            </div>
                        </a>
                    </li>
                    <li class="divider"></li>
                    <li>
                        <a href="#">
                            <div>
                                <p> <strong>Task 4</strong> <span class="pull-right text-muted">80% Complete</span> </p>
                                <div class="progress progress-striped active">
                                    <div class="progress-bar progress-bar-danger" role="progressbar" aria-valuenow="80" aria-valuemin="0" aria-valuemax="100" style="width: 80%"> <span class="sr-only">80% Complete (danger)</span> </div>
                                </div>
                            </div>
                        </a>
                    </li>
                    <li class="divider"></li>
                    <li>
                        <a class="text-center" href="#"> <strong>See All Tasks</strong> <i class="fa fa-angle-right"></i> </a>
                    </li>
                </ul>
            </li>
        </ul>
        <ul class="nav navbar-top-links navbar-right pull-right">
            <li>
                <form role="search" class="app-search hidden-sm hidden-xs m-r-10">
                    <input type="text" placeholder="Search..." class="form-control"> <a href=""><i class="fa fa-search"></i></a> </form>
            </li>
            <li class="dropdown">
                <a class="dropdown-toggle profile-pic" data-toggle="dropdown" href="#"> <img src="{{ url_for('static', filename = 'img/guest.png') }}" alt="user-img" width="36" class="img-circle"><b class="hidden-xs">{{session.name}}</b><span class="caret"></span> </a>
                <ul class="dropdown-menu dropdown-user animated flipInY">
                    <li>
                        <div class="dw-user-box">
                            <div class="u-img"><img src="{{ url_for('static', filename = 'img/guest.png') }}" alt="user" /></div>
                            <div class="u-text">
                                <h4>{{session.name}}</h4>
                                <p class="text-muted">{{session.mail}}</p><a href="profile.html" class="btn btn-rounded btn-danger btn-sm">View Profile</a></div>
                        </div>
                    </li>
                    <li role="separator" class="divider"></li>
                    <li><a href="#"><i class="ti-user"></i> My Profile</a></li>
                    <li><a href="/signout"><i class="fa fa-power-off"></i> Logout</a></li>
                </ul>
            </li>
        </ul>
    </div>
</nav>

Overwriting ./templates/include/top_header.html


In [ ]:
%%file ./templates/include/link.html
<link rel="shortcut icon" href="https://filkom.ub.ac.id/assets/icons/xfavicon-32x32.png.pagespeed.ic.aoSXDJiknB.webp" type="image/x-icon"/>
<!-- Bootstrap Core CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/bootstrap.min.css') }}" />
<!-- Menu CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/sidebar-nav.min.css') }}" />
<!-- toast CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/jquery.toast.css') }}" />
<!-- morris CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/morris.css') }}" />
<!-- chartist CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/chartist.min.css') }}" />
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/chartist-plugin-tooltip.css') }}" />
<!-- Calendar CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/fullcalendar.css') }}" />
<!-- animation CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/animate.css') }}" />
<!-- Custom CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/style.css') }}" />
<!-- Spinner CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/spinners.css') }}" />
<!-- color CSS -->
<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename = 'css/blue-dark.css') }}" id="theme" />

Overwriting ./templates/include/link.html


In [ ]:
%%file ./templates/extends/base.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <meta name="description" content="">
    <meta name="author" content="">
    {% block title %}{% endblock title %}
    {% include "include/link.html" %}
</head>
<body>
    {% include "include/preloader.html" %}
    <div id="wrapper">
        {% include "include/top_header.html" %}
        {% include "include/left_sidebar.html" %}
        <!-- ============================================================== -->
        <!-- Page Content -->
        <!-- ============================================================== -->
        <div id="page-wrapper">
            <div class="container-fluid">
                <div class="row bg-title">
                    <div class="col-lg-3 col-md-4 col-sm-4 col-xs-12">
                        <h4 class="page-title">{{ self.title() }}</h4>
                    </div>
                    <div class="col-lg-9 col-sm-8 col-md-8 col-xs-12">
                       <ol class="breadcrumb">
                            <li><a href="#">{{ self.title() }}</a></li>
                            <li class="active">{{ self.title() }}</li>
                        </ol>
                    </div>
                </div>
                {% block content %}{% endblock content %}
            </div>
            <!-- /.container-fluid -->
            {% include "include/footer.html" %}
        </div>
        <!-- ============================================================== -->
        <!-- End Page Content -->
        <!-- ============================================================== -->
    </div>
</body>
{% include "include/script.html" %}
</html>

Overwriting ./templates/extends/base.html


In [ ]:
%%file ./templates/bigdataApps.html
{% extends "extends/base.html" %}
{% block title %}
    <title>FGA Big Data Using Python</title>
{% endblock title %}
{{ self.title() }}
    Home
{{ self.title() }}
<button onclick="window.location.href='/'" class="btn btn-outline btn-rounded btn-info">
    <i class="ti-arrow-left m-l-5"></i>
    <span>Back Home</span>
</button> Week 1

{{ self.title() }}
    Week 1

{% block content %}

<div class="row" onload="myfunction()">
    <div class="col-md-3"></div>
    <div class="col-md-6">
        <div class="white-box">
            <div class="row">
                <h3 class="box-title m-b-0">Input Parameter</h3>
                <p class="text-muted m-b-30 font-13"> Prediksi Harga Kebutuhan Pokok Menggunakan Algoritme <i>Extreme Learning Machine</i> </p>
                <form class="form-horizontal" action="bigdataApps" method="POST" enctype="multipart/form-data">
                    <input type="hidden" name="csrfmiddlewaretoken" value="{{ csrf_token }}">
                    <div class="form-group">
                        <label class="col-sm-12">Upload File Dataset <sub>.csv</sub></label>
                        <div class="col-sm-12">
                            <div class="fileinput fileinput-new input-group" data-provides="fileinput">
                                <div class="form-control" data-trigger="fileinput"> <i class="glyphicon glyphicon-file fileinput-exists"></i> <span class="fileinput-filename"></span></div> <span class="input-group-addon btn btn-default btn-file"> <span class="fileinput-new">Select file</span> <span class="fileinput-exists">Change</span>
                                <input type="file" name="inputDataset"> </span> <a href="#" class="input-group-addon btn btn-default fileinput-exists" data-dismiss="fileinput">Remove</a> </div>
                        </div>
                    </div>
                    <div class="form-group">
                        <label class="col-sm-12">Banyak Fitur</label>
                        <div class="col-sm-12">
                            <select class="form-control" name="banyakFitur" id="banyak_fitur">
                                <option selected>Pilih Banyak Fitur</option>
                                <option value="2">2</option>
                                <option value="3">3</option>
                                <option value="4">4</option>
                                <option value="5">5</option>
                                <option value="6">6</option>
                                <option value="7">7</option>
                                <option value="8">8</option>
                            </select>
                        </div>
                    </div>
                    <div class="form-group">
                        <label class="col-sm-12">Banyak <i>Hidden Neuron</i></label>
                        <div class="col-sm-12">
                            <select class="form-control" name="banyakHiddenNeuron" id="banyak_hidden_neuron">
                                <option selected>Pilih Banyak <i>Hidden Neuron</i></option>
                                <option value="2">2</option>
                                <option value="3">3</option>
                                <option value="4">4</option>
                                <option value="5">5</option>
                                <option value="6">6</option>
                                <option value="7">7</option>
                                <option value="8">8</option>
                            </select>
                        </div>
                    </div>
                    <button id="submit" class="btn btn-block btn-info btn-rounded">Proses</button>
                    </div>
                </form>
            </div>
        </div>
    </div>
    <div class="col-md-3"></div>
    <div class="panel panel-info block4">
        <div class="panel-heading"> Detail Hasil Prediksi
            <div class="pull-right"><a href="#" data-perform="panel-collapse"><i class="ti-minus"></i></a> <a href="#" data-perform="panel-dismiss"><i class="ti-close"></i></a> </div>
        </div>
        <div class="panel-wrapper collapse in" aria-expanded="true">
            <div class="panel-body">
                <div class="row">
                    <div class="col-md-2">
                        <div class="table-responsive">
                            <table class="table table-hover" data-height="10">
                                <thead>
                                    <tr>
                                    <th scope="col">Data ke-</th>
                                    <th scope="col">Nilai Aktual</th>
                                    </tr>
                                </thead>
                                <tbody>
                                    {% for aktual in y_aktual %}
                                        <tr>
                                            <th scope="row" >{{ forloop.counter0 }}</th>
                                            <td>{{'%0.2f'|format(aktual|float)}}</td>
                                        </tr>
                                    {% endfor %}
                                </tbody>
                            </table>
                        </div>
                    </div>
                    <div class="col-md-2">
                        <div class="table-responsive">
                            <table class="table table-hover" data-height="10">
                                <thead>
                                    <tr>
                                    <th scope="col">Nilai Prediksi</th>
                                    </tr>
                                </thead>
                                <tbody>
                                    {% for prediksi in y_prediksi %}
                                        <tr>
                                            <td>{{'%0.2f'|format(prediksi|float)}}</td>
                                        </tr>
                                    {% endfor %}
                                </tbody>
                            </table>
                        </div>
                    </div>
                    <div class="col-md-8">
                        <div class="form-group">
                            <div class="row">
                                <label class="col-md-3" for="example-email"><h2>Nilai MAPE</h2></label>
                                <div class="col-md-9">
                                    <input value="{{mape}}" type="email" id="example-email" name="example-email" class="form-control" placeholder="Nilai MAPE" readonly>
                                </div>
                            </div>
                        </div>
                        <div class="table-responsive border tableFixHead" style="height:470px" id="hasilGrafik"></div>
                    </div>
                </div>
            </div>
        </div>
    </div>
</div>
{% endblock content %}

Overwriting ./templates/bigdataApps.html


In [ ]:
%%file ./templates/register.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/meyer-reset/2.0/reset.min.css">
    <title>Register | Flask</title>
    <!-- Latest compiled and minified CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">

    <!-- Optional theme -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap-theme.min.css" integrity="sha384-rHyoN1iRsVXV4nD0JutlnGaslCJuC7uwjduW9SVrLvRYooPp2bWYgmgJQIXwl/Sp" crossorigin="anonymous">
</head>
<body>
    <div class="container">
        <div class="row" style="margin-top: 40px">
            <div class="col-sm-6">
                <form method="POST">                    
                    <div class="form-group">
                      <label for="email">Username (Email) : </label>
                      <input type="email" name="mail" class="form-control" id="mail">
                    </div>
                    <div class="form-group">
                      <label for="email">Name : </label>
                      <input type="text" name="uname" class="form-control" id="uname">
                    </div>
                    <div class="form-group">
                      <label for="email">Password : </label>
                      <input type="password" name="passw" class="form-control" id="passw">
                    </div>
                    <button type="submit" class="btn form-control btn-default">Register</button>
                </form>
            </div>
        </div>
    </div>


<!-- Latest compiled and minified JavaScript -->
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js" integrity="sha384-Tc5IQib027qvyjSMfHjOMaLkfuWVxZxUPnCJA7l2mCWNIpG9mGCD8wGNIcPD7Txa" crossorigin="anonymous"></script>
</body>
</html>

Overwriting ./templates/register.html


In [ ]:
%%file ./templates/login.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/meyer-reset/2.0/reset.min.css">
    <title>Login | Flask</title>
    <!-- Latest compiled and minified CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">

    <!-- Optional theme -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap-theme.min.css" integrity="sha384-rHyoN1iRsVXV4nD0JutlnGaslCJuC7uwjduW9SVrLvRYooPp2bWYgmgJQIXwl/Sp" crossorigin="anonymous">
</head>
<body>
    <div class="container">
        <div class="row" style="margin-top: 40px">
            <div class="col-sm-6">
                <form method="POST">
                    <div class="form-group">
                      <label for="email">Username (Email) : </label>
                      <input type="text" name="mail" class="form-control" id="mail">
                    </div>
                    <div class="form-group">
                      <label for="email">Password : </label>
                      <input type="password" name="passw" class="form-control" id="passw">
                    </div>
                    <button type="submit" class="btn form-control btn-default">Login</button>
                </form>
            </div>
        </div>
    </div>


<!-- Latest compiled and minified JavaScript -->
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js" integrity="sha384-Tc5IQib027qvyjSMfHjOMaLkfuWVxZxUPnCJA7l2mCWNIpG9mGCD8wGNIcPD7Txa" crossorigin="anonymous"></script>
</body>
</html>

Overwriting ./templates/login.html


In [ ]:
%%file index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/meyer-reset/2.0/reset.min.css">
    <title>Login - Register Form | Flask</title>
    <!-- Latest compiled and minified CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">

    <!-- Optional theme -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap-theme.min.css" integrity="sha384-rHyoN1iRsVXV4nD0JutlnGaslCJuC7uwjduW9SVrLvRYooPp2bWYgmgJQIXwl/Sp" crossorigin="anonymous">
</head>
<body>
    <div class="container">
    <h1>Selamat Datang!</h1>
        <div class="row" style="margin-top: 40px">
            <div class="col-sm-6">
                <ul>
                    <li class="col-sm-6"><a href="/login" class="btn form-control btn-success">Login</a></li>
                    <li class="col-sm-6"><a href="/register" class="btn form-control btn-primary">Register</a></li>
                    <li class="col-sm-6"><a href="/logout" class="btn form-control btn-danger">Logout</a></li>
                </ul>
            </div>
        </div>
    </div>


<!-- Latest compiled and minified JavaScript -->
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js" integrity="sha384-Tc5IQib027qvyjSMfHjOMaLkfuWVxZxUPnCJA7l2mCWNIpG9mGCD8wGNIcPD7Txa" crossorigin="anonymous"></script>
</body>
</html>

Writing index.html


In [ ]:
from flask import Flask
import threading

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':6060}).start()

In [ ]:
import IPython.display

def display(port, height):
    shell = """
        (async () => {
            const url = await google.colab.kernel.proxyPort(%PORT%, {"cache": true});
            const iframe = document.createElement('iframe');
            iframe.src = url;
            iframe.setAttribute('width', '100%');
            iframe.setAttribute('height', '%HEIGHT%');
            iframe.setAttribute('frameborder', 0);
            document.body.appendChild(iframe);
        })();
    """
    replacements = [
        ("%PORT%", "%d" % port),
        ("%HEIGHT%", "%d" % height),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)

    script = IPython.display.Javascript(shell)
    IPython.display.display(script)

display(6060, 400)

<IPython.core.display.Javascript object>